In [ ]:
# =============================================
# [5/9] 어텐션 기법①: Bahdanau & Luong
# =============================================
# 목표: Seq2Seq의 성능을 획기적으로 개선한 두 가지 주요 어텐션 메커니즘을 구현하고 비교합니다.

# --- 1. 기본 설정 (Seq2Seq 단계와 유사) ---
!pip install torch torchtext transformers datasets spacy
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

import torch
import torch.nn as nn
import torch.nn.functional as F

# --- Seq2Seq의 Encoder와 데이터 준비 코드는 04번에서 가져와 사용한다고 가정 ---
# (04_seq2seq_basics.ipynb의 EncoderRNN, Vocab, 데이터 로딩/토크나이징 코드)
# ... (생략) ...
# EncoderRNN, SRC_VOCAB, TRG_VOCAB, DEVICE 등이 준비되었다고 가정합니다.

# --- 2. Bahdanau (Additive) Attention 모듈 구현 ---
class BahdanauAttention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        self.attn_W1 = nn.Linear(hid_dim, hid_dim)
        self.attn_W2 = nn.Linear(hid_dim, hid_dim)
        self.attn_v = nn.Linear(hid_dim, 1, bias=False)

    def forward(self, decoder_hidden, encoder_outputs):
        # decoder_hidden: [batch_size, hid_dim]
        # encoder_outputs: [src_len, batch_size, hid_dim]
        src_len = encoder_outputs.shape[0]
        
        # 디코더 히든 상태를 src_len 만큼 반복
        decoder_hidden_repeated = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        # encoder_outputs는 [src_len, batch_size, hid_dim] -> [batch_size, src_len, hid_dim]
        encoder_outputs_permuted = encoder_outputs.permute(1, 0, 2)

        # energy = v.T * tanh(W1*h_d + W2*h_e)
        energy = torch.tanh(self.attn_W1(decoder_hidden_repeated) + self.attn_W2(encoder_outputs_permuted))
        attention_scores = self.attn_v(energy).squeeze(2) # [batch_size, src_len]
        
        return F.softmax(attention_scores, dim=1)

# --- 3. 어텐션을 적용한 Decoder 구현 ---
class AttnDecoderRNN(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, attention):
        super().__init__()
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        # 어텐션 컨텍스트 벡터가 임베딩과 합쳐지므로 입력 차원이 달라짐
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim) 
        self.fc_out = nn.Linear(hid_dim, output_dim)

    def forward(self, input, decoder_hidden, encoder_outputs):
        # input: [batch_size]
        # decoder_hidden: [batch_size, hid_dim]
        # encoder_outputs: [src_len, batch_size, hid_dim]
        input = input.unsqueeze(0) # [1, batch_size]
        embedded = self.embedding(input).squeeze(0) # [batch_size, emb_dim]

        # 어텐션 가중치 및 컨텍스트 벡터 계산
        a = self.attention(decoder_hidden, encoder_outputs) # [batch_size, src_len]
        a = a.unsqueeze(1) # [batch_size, 1, src_len]
        encoder_outputs_permuted = encoder_outputs.permute(1, 0, 2) # [batch_size, src_len, hid_dim]
        
        context_vector = torch.bmm(a, encoder_outputs_permuted).squeeze(1) # [batch_size, hid_dim]
        
        # RNN 입력: 임베딩 벡터와 컨텍스트 벡터를 concat
        rnn_input = torch.cat((embedded, context_vector), dim=1)
        
        # GRU는 hidden state만 사용. [1, batch_size, hid_dim] 형태로 입력
        output, hidden = self.rnn(rnn_input.unsqueeze(0), decoder_hidden.unsqueeze(0))
        
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden.squeeze(0)

# --- 4. 모델 래핑 및 과제 안내 ---
# (실제 학습을 위해서는 Seq2Seq 모델도 어텐션을 사용하도록 수정해야 합니다.)
# hid_dim = 512
# attn = BahdanauAttention(hid_dim)
# dec = AttnDecoderRNN(OUTPUT_DIM, DEC_EMB_DIM, hid_dim, attn)
# model = Seq2Seq(enc, dec, DEVICE) # Seq2Seq 클래스도 수정 필요

print("Bahdanau 어텐션 모듈과 이를 사용한 Decoder가 정의되었습니다.")
print("\n[실습 과제]")
print("1. Luong Attention (dot, general)을 별도 모듈로 구현해보세요.")
print("   - Dot: score(h_t, h_s) = h_t^T * h_s")
print("   - General: score(h_t, h_s) = h_t^T * W_a * h_s")
print("2. 어텐션을 적용했을 때와 아닐 때의 번역 품질(BLEU 스코어)을 비교해보세요.")